## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,85.32,86,83,17.74,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,74.07,70,9,23.53,clear sky
2,2,Busselton,AU,-33.6500,115.3333,48.09,76,0,11.88,clear sky
3,3,Farah,AF,32.5000,62.5000,79.61,10,0,2.53,clear sky
4,4,Vilyuysk,RU,63.7553,121.6247,39.72,76,100,3.56,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,85.32,86,83,17.74,light rain
3,3,Farah,AF,32.5000,62.5000,79.61,10,0,2.53,clear sky
5,5,Micheweni,TZ,-4.9667,39.8333,76.69,78,7,10.38,clear sky
6,6,Sinnamary,GF,5.3833,-52.9500,87.80,59,36,11.65,scattered clouds
7,7,Uige,AO,-7.6087,15.0613,80.17,55,87,4.56,overcast clouds
15,15,Kirakira,SB,-10.4544,161.9205,77.40,86,78,6.06,broken clouds
16,16,Atuona,PF,-9.8000,-139.0333,77.70,73,4,16.91,clear sky
24,24,Diego De Almagro,CL,-26.3667,-70.0500,80.98,8,0,8.03,clear sky
25,25,Ndele,CF,8.4109,20.6473,78.04,87,100,0.94,overcast clouds
26,26,Soyo,AO,-6.1349,12.3689,76.71,78,96,8.70,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                178
City                   178
Country                177
Lat                    178
Lng                    178
Max Temp               178
Humidity               178
Cloudiness             178
Wind Speed             178
Current Description    178
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,85.32,light rain,22.2855,114.1577,
3,Farah,AF,79.61,clear sky,32.5000,62.5000,
5,Micheweni,TZ,76.69,clear sky,-4.9667,39.8333,
6,Sinnamary,GF,87.80,scattered clouds,5.3833,-52.9500,
7,Uige,AO,80.17,overcast clouds,-7.6087,15.0613,
15,Kirakira,SB,77.40,broken clouds,-10.4544,161.9205,
16,Atuona,PF,77.70,clear sky,-9.8000,-139.0333,
24,Diego De Almagro,CL,80.98,clear sky,-26.3667,-70.0500,
25,Ndele,CF,78.04,overcast clouds,8.4109,20.6473,
26,Soyo,AO,76.71,overcast clouds,-6.1349,12.3689,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,85.32,light rain,22.2855,114.1577,Mini Hotel Central
3,Farah,AF,79.61,clear sky,32.5000,62.5000,
5,Micheweni,TZ,76.69,clear sky,-4.9667,39.8333,
6,Sinnamary,GF,87.80,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
7,Uige,AO,80.17,overcast clouds,-7.6087,15.0613,Hotel Catraio
...,...,...,...,...,...,...,...
544,Cabatuan,PH,75.49,overcast clouds,10.8794,122.4860,MVC Transient Room
548,Voi,KE,76.17,overcast clouds,-3.3945,38.5630,Voi Wildlife Lodge
550,Itarema,BR,88.03,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
551,Acajutla,SV,85.84,broken clouds,13.5928,-89.8275,asi es


In [10]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,85.32,light rain,22.2855,114.1577,Mini Hotel Central
6,Sinnamary,GF,87.80,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
7,Uige,AO,80.17,overcast clouds,-7.6087,15.0613,Hotel Catraio
15,Kirakira,SB,77.40,broken clouds,-10.4544,161.9205,SanBiz Lodge
16,Atuona,PF,77.70,clear sky,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
544,Cabatuan,PH,75.49,overcast clouds,10.8794,122.4860,MVC Transient Room
548,Voi,KE,76.17,overcast clouds,-3.3945,38.5630,Voi Wildlife Lodge
550,Itarema,BR,88.03,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
551,Acajutla,SV,85.84,broken clouds,13.5928,-89.8275,asi es


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))